In [2]:
import mlflow.keras
import mlflow
import mlflow.tensorflow
from sklearn.model_selection import train_test_split, cross_val_score

from table_functions import *
from analyze_visualisation import *
from ml_models.decision_tree import *
from ml_models.random_forest import *
from ml_models.xg_boost import *
from ml_models.neural_network_train import *
from ml_models.extra_random_forest import *
from ml_functions import *

In [ ]:
# TO MUSI BYC USTAWIONE ZAWSZE TAK SAMO, a potem jak sie chce umieszczac dane modele w innym folderze to zmienia sie nazwe  artifact_directory = "other_classification"
# jak chce sie wlaczyc potem mlflow w przegladarce, trzeba w terminalu przejsc do lokalizacji 'src' i wpisac komende mlflow ui
current_working_directory = os.getcwd()
mlflow.set_tracking_uri(f"file://{os.path.join(current_working_directory, 'mlruns')}")

In [ ]:
# not normalized data
ml_data_nn = read_data_for_traning('n_n')

In [ ]:
# normalized in range 0-1
ml_data_0_1 = read_data_for_traning('0_1')

In [ ]:
# standard normalized
ml_data_ss = read_data_for_traning('s_s')

<h4>Drzewo decyzyjne</h4>

In [ ]:
clf = decision_tree_model(*ml_data_ss.values(), run_name_="normal_run")

<h4>Las losowy<h4>

In [ ]:
clf = random_forest_model(*ml_data_ss.values(), run_name_="only_4_columns_1")

<h4>XGBoost</h4>

In [ ]:
clf = xgb_model(*ml_data_ss.values(), run_name_="test_droped_0_1")

<h4>Sieci neuronowe<h4>

In [ ]:
compile_fit_evaluate_model(*ml_data_ss.values(), epochs_=2, batch_size_=512, model_number=1, run_name_=f"data__droped_save_fig")

<h4>Super drzewa losowe</h4>

In [ ]:
clf = extra_random_trees(*ml_data_ss.values(), run_name_="first_run")

<h4>Cross Validacja</h4>

Według mnie na razie zostawmy cross validacje w spokoju, na normalnym zbiorze znajdzmy najlepsze parametry dla modeli a potem dopiero zrobmy na takich parametrach cross validacje by zobaczyc 'prawdziwa' jakosc modelu
PS. ponizszy kod wykonuje sie jakas 1h
PS2. dane sie zapisaly ale jakis syf

In [ ]:
artifact_directory="cv_random_forest"
mlflow.set_experiment(artifact_directory)
mlflow.sklearn.autolog()

with start_run(run_name="testowy run"):
    clf = RandomForestClassifier(random_state=0)
    scores = cross_val_score(estimator=clf, X=ml_data_ss['x_train'], y=ml_data_ss['y_train'], cv=5)
print(scores)
# [0.99786842 0.99779825 0.99742105 0.99770175 0.99797368] - wynik

In [ ]:
loaded_model = mlflow.keras.load_model(r'file://C:\Users\dlxpmx8\Desktop\Projekt_AI\meb_process_data_analysis\src\mlruns/386163800829212723/5e018f9f70514ad889d242edd0430b67/artifacts/model')

In [ ]:
distribution_of_probability(ml_data['x_test'], ml_data['y_test'], model_=loaded_model)